# Importaciones


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab

/content/drive/MyDrive/Colab


In [ ]:
!pip install requests
!pip install unidecode
!sudo apt-get install -y tesseract-ocr
!pip install pillow
!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,284 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program i

In [ ]:
# IMPORTACIÓN DE LIBRERÍAS
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import numpy as np
import re
import matplotlib.pyplot as plt
import requests
import json
from geopandas import GeoDataFrame as gpd
from shapely.geometry import Polygon
from zipfile import ZipFile
from shutil import rmtree

# Código de consulta

In [ ]:
# DATA MUNICIPIOS
data = {'MEDELLIN': '5001', 'ABEJORRAL': '5002', 'ABRIAQUI': '5004', 'ALEJANDRIA': '5021', 'AMAGA': '5030', 'AMALFI': '5031', 'ANDES': '5034', 'ANGELOPOLIS': '5036', 'ANGOSTURA': '5038', 'ANORI': '5040', 'SANTAFE DE ANTIOQUIA': '5042', 'ANZA': '5044', 'APARTADO': '5045', 'ARBOLETES': '5051', 'ARGELIA': '76054', 'ARMENIA': '63001', 'BARBOSA': '68077', 'BELMIRA': '5086', 'BELLO': '5088', 'BETANIA': '5091', 'BETULIA': '68092', 'CIUDAD BOLIVAR': '5101', 'BRICENO': '15106', 'BURITICA': '5113', 'CACERES': '5120', 'CAICEDO': '5125', 'CALDAS': '15131', 'CAMPAMENTO': '5134', 'CANASGORDAS': '5138', 'CARACOLI': '5142', 'CARAMANTA': '5145', 'CAREPA': '5147', 'EL CARMEN DE VIBORAL': '5148', 'CAROLINA': '5150', 'CAUCASIA': '5154', 'CHIGORODO': '5172', 'CISNEROS': '5190', 'COCORNA': '5197', 'CONCEPCION': '68207', 'CONCORDIA': '47205', 'COPACABANA': '5212', 'DABEIBA': '5234', 'DON MATIAS': '5237', 'EBEJICO': '5240', 'EL BAGRE': '5250', 'ENTRERRIOS': '5264', 'ENVIGADO': '5266', 'FREDONIA': '5282', 'FRONTINO': '5284', 'GIRALDO': '5306', 'GIRARDOTA': '5308', 'GOMEZ PLATA': '5310', 'GRANADA': '50313', 'GUADALUPE': '68320', 'GUARNE': '5318', 'GUATAPE': '5321', 'HELICONIA': '5347', 'HISPANIA': '5353', 'ITAGUI': '5360', 'ITUANGO': '5361', 'JARDIN': '5364', 'JERICO': '15368', 'LA CEJA': '5376', 'LA ESTRELLA': '5380', 'LA PINTADA': '5390', 'LA UNION': '76400', 'LIBORINA': '5411', 'MACEO': '5425', 'MARINILLA': '5440', 'MONTEBELLO': '5467', 'MURINDO': '5475', 'MUTATA': '5480', 'NARINO': '52480', 'NECOCLI': '5490', 'NECHI': '5495', 'OLAYA': '5501', 'PENOL': '5541', 'PEQUE': '5543', 'PUEBLORRICO': '5576', 'PUERTO BERRIO': '5579', 'PUERTO NARE': '5585', 'PUERTO TRIUNFO': '5591', 'REMEDIOS': '5604', 'RETIRO': '5607', 'RIONEGRO': '68615', 'SABANALARGA': '85300', 'SABANETA': '5631', 'SALGAR': '5642', 'SAN ANDRES DE CUERQUIA': '5647', 'SAN CARLOS': '23678', 'SAN FRANCISCO': '86755', 'SAN JERONIMO': '5656', 'SAN JOSE DE LA MONTANA': '5658', 'SAN JUAN DE URABA': '5659', 'SAN LUIS': '73678', 'SAN PEDRO': '76670', 'SAN PEDRO DE URABA': '5665', 'SAN RAFAEL': '5667', 'SAN ROQUE': '5670', 'SAN VICENTE': '5674', 'SANTA BARBARA': '68705', 'SANTA ROSA DE OSOS': '5686', 'SANTO DOMINGO': '5690', 'EL SANTUARIO': '5697', 'SEGOVIA': '5736', 'SONSON': '5756', 'SOPETRAN': '5761', 'TAMESIS': '5789', 'TARAZA': '5790', 'TARSO': '5792', 'TITIRIBI': '5809', 'TOLEDO': '54820', 'TURBO': '5837', 'URAMITA': '5842', 'URRAO': '5847', 'VALDIVIA': '5854', 'VALPARAISO': '18860', 'VEGACHI': '5858', 'VENECIA': '25506', 'VIGIA DEL FUERTE': '5873', 'YALI': '5885', 'YARUMAL': '5887', 'YOLOMBO': '5890', 'YONDO': '5893', 'ZARAGOZA': '5895', 'BARRANQUILLA': '8001', 'BARANOA': '8078', 'CAMPO DE LA CRUZ': '8137', 'CANDELARIA': '76130', 'GALAPA': '8296', 'JUAN DE ACOSTA': '8372', 'LURUACO': '8421', 'MALAMBO': '8433', 'MANATI': '8436', 'PALMAR DE VARELA': '8520', 'PIOJO': '8549', 'POLONUEVO': '8558', 'PONEDERA': '8560', 'PUERTO COLOMBIA': '94884', 'REPELON': '8606', 'SABANAGRANDE': '8634', 'SANTA LUCIA': '8675', 'SANTO TOMAS': '8685', 'SOLEDAD': '8758', 'SUAN': '8770', 'TUBARA': '8832', 'USIACURI': '8849', 'BOGOTA, D.C.': '11001', 'CARTAGENA': '13001', 'ACHI': '13006', 'ALTOS DEL ROSARIO': '13030', 'ARENAL': '13042', 'ARJONA': '13052', 'ARROYOHONDO': '13062', 'BARRANCO DE LOBA': '13074', 'CALAMAR': '95015', 'CANTAGALLO': '13160', 'CICUCO': '13188', 'CORDOBA': '63212', 'CLEMENCIA': '13222', 'EL CARMEN DE BOLIVAR': '13244', 'EL GUAMO': '13248', 'EL PENON': '68250', 'HATILLO DE LOBA': '13300', 'MAGANGUE': '13430', 'MAHATES': '13433', 'MARGARITA': '13440', 'MARIA LA BAJA': '13442', 'MONTECRISTO': '13458', 'MOMPOS': '13468', 'MORALES': '19473', 'PINILLOS': '13549', 'REGIDOR': '13580', 'RIO VIEJO': '13600', 'SAN CRISTOBAL': '13620', 'SAN ESTANISLAO': '13647', 'SAN FERNANDO': '13650', 'SAN JACINTO': '13654', 'SAN JACINTO DEL CAUCA': '13655', 'SAN JUAN NEPOMUCENO': '13657', 'SAN MARTIN DE LOBA': '13667', 'SAN PABLO': '52693', 'SANTA CATALINA': '13673', 'SANTA ROSA': '19701', 'SANTA ROSA DEL SUR': '13688', 'SIMITI': '13744', 'SOPLAVIENTO': '13760', 'TALAIGUA NUEVO': '13780', 'TIQUISIO': '13810', 'TURBACO': '13836', 'TURBANA': '13838', 'VILLANUEVA': '85440', 'ZAMBRANO': '13894', 'TUNJA': '15001', 'ALMEIDA': '15022', 'AQUITANIA': '15047', 'ARCABUCO': '15051', 'BELEN': '52083', 'BERBEO': '15090', 'BETEITIVA': '15092', 'BOAVITA': '15097', 'BOYACA': '15104', 'BUENAVISTA': '70110', 'BUSBANZA': '15114', 'CAMPOHERMOSO': '15135', 'CERINZA': '15162', 'CHINAVITA': '15172', 'CHIQUINQUIRA': '15176', 'CHISCAS': '15180', 'CHITA': '15183', 'CHITARAQUE': '15185', 'CHIVATA': '15187', 'CIENEGA': '15189', 'COMBITA': '15204', 'COPER': '15212', 'CORRALES': '15215', 'COVARACHIA': '15218', 'CUBARA': '15223', 'CUCAITA': '15224', 'CUITIVA': '15226', 'CHIQUIZA': '15232', 'CHIVOR': '15236', 'DUITAMA': '15238', 'EL COCUY': '15244', 'EL6aSPINO': '15248', 'FIRAVITOBA': '15272', 'FLORESTA': '15276', 'GACHANTIVA': '15293', 'GAMEZA': '15296', 'GARAGOA': '15299', 'GUACAMAYAS': '15317', 'GUATEQUE': '15322', 'GUAYATA': '15325', 'GUICAN': '15332', 'IZA': '15362', 'JENESANO': '15367', 'LABRANZAGRANDE': '15377', 'LA CAPILLA': '15380', 'LA VICTORIA': '91430', 'LA UVITA': '15403', 'VILLA DE LEYVA': '15407', 'MACANAL': '15425', 'MARIPI': '15442', 'MIRAFLORES': '95200', 'MONGUA': '15464', 'MONGUI': '15466', 'MONIQUIRA': '15469', 'MOTAVITA': '15476', 'MUZO': '15480', 'NOBSA': '15491', 'NUEVO COLON': '15494', 'OICATA': '15500', 'OTANCHE': '15507', 'PACHAVITA': '15511', 'PAEZ': '19517', 'PAIPA': '15516', 'PAJARITO': '15518', 'PANQUEBA': '15522', 'PAUNA': '15531', 'PAYA': '15533', 'PAZ DE RIO': '15537', 'PESCA': '15542', 'PISBA': '15550', 'PUERTO BOYACA': '15572', 'QUIPAMA': '15580', 'RAMIRIQUI': '15599', 'RAQUIRA': '15600', 'RONDON': '15621', 'SABOYA': '15632', 'SACHICA': '15638', 'SAMACA': '15646', 'SAN EDUARDO': '15660', 'SAN JOSE DE PARE': '15664', 'SAN LUIS DE GACENO': '15667', 'SAN MATEO': '15673', 'SAN MIGUEL DE SEMA': '15676', 'SAN PABLO DE BORBUR': '15681', 'SANTANA': '15686', 'SANTA MARIA': '41676', 'SANTA ROSA DE VITERBO': '15693', 'SANTA SOFIA': '15696', 'SATIVANORTE': '15720', 'SATIVASUR': '15723', 'SIACHOQUE': '15740', 'SOATA': '15753', 'SOCOTA': '15755', 'SOCHA': '15757', 'SOGAMOSO': '15759', 'SOMONDOCO': '15761', 'SORA': '15762', 'SOTAQUIRA': '15763', 'SORACA': '15764', 'SUSACON': '15774', 'SUTAMARCHAN': '15776', 'SUTATENZA': '15778', 'TASCO': '15790', 'TENZA': '15798', 'TIBANA': '15804', 'TIBASOSA': '15806', 'TINJACA': '15808', 'TIPACOQUE': '15810', 'TOCA': '15814', 'TOGUI': '15816', 'TOPAGA': '15820', 'TOTA': '15822', 'TUNUNGUA': '15832', 'TURMEQUE': '15835', 'TUTA': '15837', 'TUTAZA': '15839', 'UMBITA': '15842', 'VENTAQUEMADA': '15861', 'VIRACACHA': '15879', 'ZETAQUIRA': '15897', 'MANIZALES': '17001', 'AGUADAS': '17013', 'ANSERMA': '17042', 'ARANZAZU': '17050', 'BELALCAZAR': '17088', 'CHINCHINA': '17174', 'FILADELFIA': '17272', 'LA DORADA': '17380', 'LA MERCED': '17388', 'MANZANARES': '17433', 'MARMATO': '17442', 'MARQUETALIA': '17444', 'MARULANDA': '17446', 'NEIRA': '17486', 'NORCASIA': '17495', 'PACORA': '17513', 'PALESTINA': '41530', 'PENSILVANIA': '17541', 'RIOSUCIO': '27615', 'RISARALDA': '17616', 'SALAMINA': '47675', 'SAMANA': '17662', 'SAN JOSE': '17665', 'SUPIA': '17777', 'VICTORIA': '17867', 'VILLAMARIA': '17873', 'VITERBO': '17877', 'FLORENCIA': '19290', 'ALBANIA': '68020', 'BELEN DE LOS ANDAQUIES': '18094', 'CARTAGENA DEL CHAIRA': '18150', 'CURILLO': '18205', 'EL DONCELLO': '18247', 'EL PAUJIL': '18256', 'LA MONTANITA': '18410', 'MILAN': '18460', 'MORELIA': '18479', 'PUERTO RICO': '50590', 'SAN JOSE DEL FRAGUA': '18610', 'SAN VICENTE DEL CAGUAN': '18753', 'SOLANO': '18756', 'SOLITA': '18785', 'POPAYAN': '19001', 'ALMAGUER': '19022', 'BALBOA': '66075', 'BOLIVAR': '76100', 'BUENOS AIRES': '19110', 'CAJIBIO': '19130', 'CALDONO': '19137', 'CALOTO': '19142', 'CORINTO': '19212', 'EL TAMBO': '52260', 'GUACHENE': '19300', 'GUAPI': '19318', 'INZA': '19355', 'JAMBALO': '19364', 'LA SIERRA': '19392', 'LA VEGA': '25402', 'LOPEZ': '19418', 'MERCADERES': '19450', 'MIRANDA': '19455', 'PADILLA': '19513', 'PATIA': '19532', 'PIAMONTE': '19533', 'PIENDAMO': '19548', 'PUERTO TEJADA': '19573', 'PURACE': '19585', 'ROSAS': '19622', 'SAN SEBASTIAN': '19693', 'SANTANDER DE QUILICHAO': '19698', 'SILVIA': '19743', 'SOTARA': '19760', 'SUAREZ': '73770', 'SUCRE': '70771', 'TIMBIO': '19807', 'TIMBIQUI': '19809', 'TORIBIO': '19821', 'TOTORO': '19824', 'VILLA RICA': '19845', 'VALLEDUPAR': '20001', 'AGUACHICA': '20011', 'AGUSTIN CODAZZI': '20013', 'ASTREA': '20032', 'BECERRIL': '20045', 'BOSCONIA': '20060', 'CHIMICHAGUA': '20175', 'CHIRIGUANA': '20178', 'CURUMANI': '20228', 'EL COPEY': '20238', 'EL PASO': '20250', 'GAMARRA': '20295', 'GONZALEZ': '20310', 'LA GLORIA': '20383', 'LA JAGUA DE IBIRICO': '20400', 'MANAURE': '44560', 'PAILITAS': '20517', 'PELAYA': '20550', 'PUEBLO BELLO': '20570', 'RIO DE ORO': '20614', 'LA PAZ': '68397', 'SAN ALBERTO': '20710', 'SAN DIEGO': '20750', 'SAN MARTIN': '50689', 'TAMALAMEQUE': '20787', 'MONTERIA': '23001', 'AYAPEL': '23068', 'CANALETE': '23090', 'CERETE': '23162', 'CHIMA': '68176', 'CHINU': '23182', 'CIENAGA DE ORO': '23189', 'COTORRA': '23300', 'LA APARTADA': '23350', 'LORICA': '23417', 'LOS CORDOBAS': '23419', 'MOMIL': '23464', 'MONTELIBANO': '23466', 'MONITOS': '23500', 'PLANETA RICA': '23555', 'PUEBLO NUEVO': '23570', 'PUERTO ESCONDIDO': '23574', 'PUERTO LIBERTADOR': '23580', 'PURISIMA': '23586', 'SAHAGUN': '23660', 'SAN ANDRES SOTAVENTO': '23670', 'SAN ANTERO': '23672', 'SAN BERNARDO DEL VIENTO': '23675', 'SAN PELAYO': '23686', 'TIERRALTA': '23807', 'VALENCIA': '23855', 'AGUA DE DIOS': '25001', 'ALBAN': '52019', 'ANAPOIMA': '25035', 'ANOLAIMA': '25040', 'ARBELAEZ': '25053', 'BELTRAN': '25086', 'BITUIMA': '25095', 'BOJACA': '25099', 'CABRERA': '68121', 'CACHIPAY': '25123', 'CAJICA': '25126', 'CAPARRAPI': '25148', 'CAQUEZA': '25151', 'CARMEN DE CARUPA': '25154', 'CHAGUANI': '25168', 'CHIA': '25175', 'CHIPAQUE': '25178', 'CHOACHI': '25181', 'CHOCONTA': '25183', 'COGUA': '25200', 'COTA': '25214', 'CUCUNUBA': '25224', 'EL COLEGIO': '25245', 'EL ROSAL': '25260', 'FACATATIVA': '25269', 'FOMEQUE': '25279', 'FOSCA': '25281', 'FUNZA': '25286', 'FUQUENE': '25288', 'FUSAGASUGA': '25290', 'GACHALA': '25293', 'GACHANCIPA': '25295', 'GACHETA': '25297', 'GAMA': '25299', 'GIRARDOT': '25307', 'GUACHETA': '25317', 'GUADUAS': '25320', 'GUASCA': '25322', 'GUATAQUI': '25324', 'GUATAVITA': '25326', 'GUAYABAL DE SIQUIMA': '25328', 'GUAYABETAL': '25335', 'GUTIERREZ': '25339', 'JERUSALEN': '25368', 'JUNIN': '25372', 'LA CALERA': '25377', 'LA MESA': '25386', 'LA PALMA': '25394', 'LA PENA': '25398', 'LENGUAZAQUE': '25407', 'MACHETA': '25426', 'MADRID': '25430', 'MANTA': '25436', 'MEDINA': '25438', 'MOSQUERA': '52473', 'NEMOCON': '25486', 'NILO': '25488', 'NIMAIMA': '25489', 'NOCAIMA': '25491', 'PACHO': '25513', 'PAIME': '25518', 'PANDI': '25524', 'PARATEBUENO': '25530', 'PASCA': '25535', 'PUERTO SALGAR': '25572', 'PULI': '25580', 'QUEBRADANEGRA': '25592', 'QUETAME': '25594', 'QUIPILE': '25596', 'APULO': '25599', 'RICAURTE': '52612', 'SAN ANTONIO DEL TEQUENDAMA': '25645', 'SAN BERNARDO': '52685', 'SAN CAYETANO': '54673', 'SAN JUAN DE RIO SECO': '25662', 'SASAIMA': '25718', 'SESQUILE': '25736', 'SIBATE': '25740', 'SILVANIA': '25743', 'SIMIJACA': '25745', 'SOACHA': '25754', 'SOPO': '25758', 'SUBACHOQUE': '25769', 'SUESCA': '25772', 'SUPATA': '25777', 'SUSA': '25779', 'SUTATAUSA': '25781', 'TABIO': '25785', 'TAUSA': '25793', 'TENA': '25797', 'TENJO': '25799', 'TIBACUY': '25805', 'TIBIRITA': '25807', 'TOCAIMA': '25815', 'TOCANCIPA': '25817', 'TOPAIPI': '25823', 'UBALA': '25839', 'UBAQUE': '25841', 'VILLA DE SAN DIEGO DE UBATE': '25843', 'UNE': '25845', 'UTICA': '25851', 'VERGARA': '25862', 'VIANI': '25867', 'VILLAGOMEZ': '25871', 'VILLAPINZON': '25873', 'VILLETA': '25875', 'VIOTA': '25878', 'YACOPI': '25885', 'ZIPACON': '25898', 'ZIPAQUIRA': '25899', 'QUIBDO': '27001', 'ACANDI': '27006', 'ALTO BAUDO': '27025', 'ATRATO': '27050', 'BAGADO': '27073', 'BAHIA SOLANO': '27075', 'BAJO BAUDO': '27077', 'BELEN DE BAJIRA': '27086', 'BOJAYA': '27099', 'EL CANTON DEL SAN PABLO': '27135', 'CARMEN DEL DARIEN': '27150', 'CERTEGUI': '27160', 'CONDOTO': '27205', 'EL CARMEN DE ATRATO': '27245', 'EL LITORAL DEL SAN JUAN': '27250', 'ISTMINA': '27361', 'JURADO': '27372', 'LLORO': '27413', 'MEDIO ATRATO': '27425', 'MEDIO BAUDO': '27430', 'MEDIO SAN JUAN': '27450', 'NOVITA': '27491', 'NUQUI': '27495', 'RIO IRO': '27580', 'RIO QUITO': '27600', 'SAN JOSE DEL PALMAR': '27660', 'SIPI': '27745', 'TADO': '27787', 'UNGUIA': '27800', 'UNION PANAMERICANA': '27810', 'NEIVA': '41001', 'ACEVEDO': '41006', 'AGRADO': '41013', 'AIPE': '41016', 'ALGECIRAS': '41020', 'ALTAMIRA': '41026', 'BARAYA': '41078', 'CAMPOALEGRE': '41132', 'COLOMBIA': '41206', 'ELIAS': '41244', 'GARZON': '41298', 'GIGANTE': '41306', 'HOBO': '41349', 'IQUIRA': '41357', 'ISNOS': '41359', 'LA ARGENTINA': '41378', 'LA PLATA': '41396', 'NATAGA': '41483', 'OPORAPA': '41503', 'PAICOL': '41518', 'PALERMO': '41524', 'PITAL': '41548', 'PITALITO': '41551', 'RIVERA': '41615', 'SALADOBLANCO': '41660', 'SAN AGUSTIN': '41668', 'SUAZA': '41770', 'TARQUI': '41791', 'TESALIA': '41797', 'TELLO': '41799', 'TERUEL': '41801', 'TIMANA': '41807', 'VILLAVIEJA': '41872', 'YAGUARA': '41885', 'RIOHACHA': '44001', 'BARRANCAS': '44078', 'DIBULLA': '44090', 'DISTRACCION': '44098', 'EL MOLINO': '44110', 'FONSECA': '44279', 'HATONUEVO': '44378', 'LA JAGUA DEL PILAR': '44420', 'MAICAO': '44430', 'SAN JUAN DEL CESAR': '44650', 'URIBIA': '44847', 'URUMITA': '44855', 'SANTA MARTA': '47001', 'ALGARROBO': '47030', 'ARACATACA': '47053', 'ARIGUANI': '47058', 'CERRO SAN ANTONIO': '47161', 'CHIBOLO': '47170', 'CIENAGA': '47189', 'EL BANCO': '47245', 'EL PINON': '47258', 'EL RETEN': '47268', 'FUNDACION': '47288', 'GUAMAL': '50318', 'NUEVA GRANADA': '47460', 'PEDRAZA': '47541', 'PIJINO DEL CARMEN': '47545', 'PIVIJAY': '47551', 'PLATO': '47555', 'PUEBLOVIEJO': '47570', 'REMOLINO': '47605', 'SABANAS DE SAN ANGEL': '47660', 'SAN SEBASTIAN DE BUENAVISTA': '47692', 'SAN ZENON': '47703', 'SANTA ANA': '47707', 'SANTA BARBARA DE PINTO': '47720', 'SITIONUEVO': '47745', 'TENERIFE': '47798', 'ZAPAYAN': '47960', 'ZONA BANANERA': '47980', 'VILLAVICENCIO': '50001', 'ACACIAS': '50006', 'BARRANCA DE UPIA': '50110', 'CABUYARO': '50124', 'CASTILLA LA NUEVA': '50150', 'CUBARRAL': '50223', 'CUMARAL': '50226', 'EL CALVARIO': '50245', 'EL CASTILLO': '50251', 'EL DORADO': '50270', 'FUENTE DE ORO': '50287', 'MAPIRIPAN': '50325', 'MESETAS': '50330', 'LA MACARENA': '50350', 'URIBE': '50370', 'LEJANIAS': '50400', 'PUERTO CONCORDIA': '50450', 'PUERTO GAITAN': '50568', 'PUERTO LOPEZ': '50573', 'PUERTO LLERAS': '50577', 'RESTREPO': '76606', 'SAN CARLOS DE GUAROA': '50680', 'SAN JUAN DE ARAMA': '50683', 'SAN JUANITO': '50686', 'VISTAHERMOSA': '50711', 'PASTO': '52001', 'ALDANA': '52022', 'ANCUYA': '52036', 'ARBOLEDA': '52051', 'BARBACOAS': '52079', 'BUESACO': '52110', 'COLON': '86219', 'CONSACA': '52207', 'CONTADERO': '52210', 'CUASPUD': '52224', 'CUMBAL': '52227', 'CUMBITARA': '52233', 'CHACHAGUI': '52240', 'EL CHARCO': '52250', 'EL PENOL': '52254', 'EL ROSARIO': '52256', 'EL TABLON DE GOMEZ': '52258', 'FUNES': '52287', 'GUACHUCAL': '52317', 'GUAITARILLA': '52320', 'GUALMATAN': '52323', 'ILES': '52352', 'IMUES': '52354', 'IPIALES': '52356', 'LA CRUZ': '52378', 'LA FLORIDA': '52381', 'LA LLANADA': '52385', 'LA TOLA': '52390', 'LEIVA': '52405', 'LINARES': '52411', 'LOS ANDES': '52418', 'MAGUI': '52427', 'MALLAMA': '52435', 'OLAYA HERRERA': '52490', 'OSPINA': '52506', 'FRANCISCO PIZARRO': '52520', 'POLICARPA': '52540', 'POTOSI': '52560', 'PROVIDENCIA': '88564', 'PUERRES': '52573', 'PUPIALES': '52585', 'ROBERTO PAYAN': '52621', 'SAMANIEGO': '52678', 'SANDONA': '52683', 'SAN LORENZO': '52687', 'SAN PEDRO DE CARTAGO': '52694', 'SANTACRUZ': '52699', 'SAPUYES': '52720', 'TAMINANGO': '52786', 'TANGUA': '52788', 'SAN ANDRES DE TUMACO': '52835', 'TUQUERRES': '52838', 'YACUANQUER': '52885', 'CUCUTA': '54001', 'ABREGO': '54003', 'ARBOLEDAS': '54051', 'BOCHALEMA': '54099', 'BUCARASICA': '54109', 'CACOTA': '54125', 'CACHIRA': '54128', 'CHINACOTA': '54172', 'CHITAGA': '54174', 'CONVENCION': '54206', 'CUCUTILLA': '54223', 'DURANIA': '54239', 'EL CARMEN': '54245', 'EL TARRA': '54250', 'EL ZULIA': '54261', 'GRAMALOTE': '54313', 'HACARI': '54344', 'HERRAN': '54347', 'LABATECA': '54377', 'LA ESPERANZA': '54385', 'LA PLAYA': '54398', 'LOS PATIOS': '54405', 'LOURDES': '54418', 'MUTISCUA': '54480', 'OCANA': '54498', 'PAMPLONA': '54518', 'PAMPLONITA': '54520', 'PUERTO SANTANDER': '91669', 'RAGONVALIA': '54599', 'SALAZAR': '54660', 'SAN CALIXTO': '54670', 'SANTIAGO': '86760', 'SARDINATA': '54720', 'SILOS': '54743', 'TEORAMA': '54800', 'TIBU': '54810', 'VILLA CARO': '54871', 'VILLA DEL ROSARIO': '54874', 'CALARCA': '63130', 'CIRCASIA': '63190', 'FILANDIA': '63272', 'GENOVA': '63302', 'LA TEBAIDA': '63401', 'MONTENEGRO': '63470', 'PIJAO': '63548', 'QUIMBAYA': '63594', 'SALENTO': '63690', 'PEREIRA': '66001', 'APIA': '66045', 'BELEN DE UMBRIA': '66088', 'DOSQUEBRADAS': '66170', 'GUATICA': '66318', 'LA CELIA': '66383', 'LA VIRGINIA': '66400', 'MARSELLA': '66440', 'MISTRATO': '66456', 'PUEBLO RICO': '66572', 'QUINCHIA': '66594', 'SANTA ROSA DE CABAL': '66682', 'SANTUARIO': '66687', 'BUCARAMANGA': '68001', 'AGUADA': '68013', 'ARATOCA': '68051', 'BARICHARA': '68079', 'BARRANCABERMEJA': '68081', 'CALIFORNIA': '68132', 'CAPITANEJO': '68147', 'CARCASI': '68152', 'CEPITA': '68160', 'CERRITO': '68162', 'CHARALA': '68167', 'CHARTA': '68169', 'CHIPATA': '68179', 'CIMITARRA': '68190', 'CONFINES': '68209', 'CONTRATACION': '68211', 'COROMORO': '68217', 'CURITI': '68229', 'EL CARMEN DE CHUCURI': '68235', 'EL GUACAMAYO': '68245', 'EL PLAYON': '68255', 'ENCINO': '68264', 'ENCISO': '68266', 'FLORIAN': '68271', 'FLORIDABLANCA': '68276', 'GALAN': '68296', 'GAMBITA': '68298', 'GIRON': '68307', 'GUACA': '68318', 'GUAPOTA': '68322', 'GUAVATA': '68324', 'GUEPSA': '68327', 'HATO': '68344', 'JESUS MARIA': '68368', 'JORDAN': '68370', 'LA BELLEZA': '68377', 'LANDAZURI': '68385', 'LEBRIJA': '68406', 'LOS SANTOS': '68418', 'MACARAVITA': '68425', 'MALAGA': '68432', 'MATANZA': '68444', 'MOGOTES': '68464', 'MOLAGAVITA': '68468', 'OCAMONTE': '68498', 'OIBA': '68500', 'ONZAGA': '68502', 'PALMAR': '68522', 'PALMAS DEL SOCORRO': '68524', 'PARAMO': '68533', 'PIEDECUESTA': '68547', 'PINCHOTE': '68549', 'PUENTE NACIONAL': '68572', 'PUERTO PARRA': '68573', 'PUERTO WILCHES': '68575', 'SABANA DE TORRES': '68655', 'SAN ANDRES': '88001', 'SAN BENITO': '68673', 'SAN GIL': '68679', 'SAN JOAQUIN': '68682', 'SAN JOSE DE MIRANDA': '68684', 'SAN MIGUEL': '86757', 'SAN VICENTE DE CHUCURI': '68689', 'SANTA HELENA DEL OPON': '68720', 'SIMACOTA': '68745', 'SOCORRO': '68755', 'SUAITA': '68770', 'SURATA': '68780', 'TONA': '68820', 'VALLE DE SAN JOSE': '68855', 'VELEZ': '68861', 'VETAS': '68867', 'ZAPATOCA': '68895', 'SINCELEJO': '70001', 'CAIMITO': '70124', 'COLOSO': '70204', 'COROZAL': '70215', 'COVENAS': '70221', 'CHALAN': '70230', 'EL ROBLE': '70233', 'GALERAS': '70235', 'GUARANDA': '70265', 'LOS PALMITOS': '70418', 'MAJAGUAL': '70429', 'MORROA': '70473', 'OVEJAS': '70508', 'PALMITO': '70523', 'SAMPUES': '70670', 'SAN BENITO ABAD': '70678', 'SAN JUAN DE BETULIA': '70702', 'SAN MARCOS': '70708', 'SAN ONOFRE': '70713', 'SAN LUIS DE SINCE': '70742', 'SANTIAGO DE TOLU': '70820', 'TOLU VIEJO': '70823', 'IBAGUE': '73001', 'ALPUJARRA': '73024', 'ALVARADO': '73026', 'AMBALEMA': '73030', 'ANZOATEGUI': '73043', 'ARMERO': '73055', 'ATACO': '73067', 'CAJAMARCA': '73124', 'CARMEN DE APICALA': '73148', 'CASABIANCA': '73152', 'CHAPARRAL': '73168', 'COELLO': '73200', 'COYAIMA': '73217', 'CUNDAY': '73226', 'DOLORES': '73236', 'ESPINAL': '73268', 'FALAN': '73270', 'FLANDES': '73275', 'FRESNO': '73283', 'GUAMO': '73319', 'HERVEO': '73347', 'HONDA': '73349', 'ICONONZO': '73352', 'LERIDA': '73408', 'LIBANO': '73411', 'MARIQUITA': '73443', 'MELGAR': '73449', 'MURILLO': '73461', 'NATAGAIMA': '73483', 'ORTEGA': '73504', 'PALOCABILDO': '73520', 'PIEDRAS': '73547', 'PLANADAS': '73555', 'PRADO': '73563', 'PURIFICACION': '73585', 'RIOBLANCO': '73616', 'RONCESVALLES': '73622', 'ROVIRA': '73624', 'SALDANA': '73671', 'SAN ANTONIO': '73675', 'SANTA ISABEL': '73686', 'VALLE DE SAN JUAN': '73854', 'VENADILLO': '73861', 'VILLAHERMOSA': '73870', 'VILLARRICA': '73873', 'CALI': '76001', 'ALCALA': '76020', 'ANDALUCIA': '76036', 'ANSERMANUEVO': '76041', 'BUENAVENTURA': '76109', 'GUADALAJARA DE BUGA': '76111', 'BUGALAGRANDE': '76113', 'CAICEDONIA': '76122', 'CALIMA': '76126', 'CARTAGO': '76147', 'DAGUA': '76233', 'EL AGUILA': '76243', 'EL CAIRO': '76246', 'EL CERRITO': '76248', 'EL DOVIO': '76250', 'FLORIDA': '76275', 'GINEBRA': '76306', 'GUACARI': '76318', 'JAMUNDI': '76364', 'LA CUMBRE': '76377', 'OBANDO': '76497', 'PALMIRA': '76520', 'PRADERA': '76563', 'RIOFRIO': '76616', 'ROLDANILLO': '76622', 'SEVILLA': '76736', 'TORO': '76823', 'TRUJILLO': '76828', 'TULUA': '76834', 'ULLOA': '76845', 'VERSALLES': '76863', 'VIJES': '76869', 'YOTOCO': '76890', 'YUMBO': '76892', 'ZARZAL': '76895', 'ARAUCA': '81001', 'ARAUQUITA': '81065', 'CRAVO NORTE': '81220', 'FORTUL': '81300', 'PUERTO RONDON': '81591', 'SARAVENA': '81736', 'TAME': '81794', 'YOPAL': '85001', 'AGUAZUL': '85010', 'CHAMEZA': '85015', 'HATO COROZAL': '85125', 'LA SALINA': '85136', 'MANI': '85139', 'MONTERREY': '85162', 'NUNCHIA': '85225', 'OROCUE': '85230', 'PAZ DE ARIPORO': '85250', 'PORE': '85263', 'RECETOR': '85279', 'SACAMA': '85315', 'SAN LUIS DE PALENQUE': '85325', 'TAMARA': '85400', 'TAURAMENA': '85410', 'TRINIDAD': '85430', 'MOCOA': '86001', 'ORITO': '86320', 'PUERTO ASIS': '86568', 'PUERTO CAICEDO': '86569', 'PUERTO GUZMAN': '86571', 'LEGUIZAMO': '86573', 'SIBUNDOY': '86749', 'VALLE DEL GUAMUEZ': '86865', 'VILLAGARZON': '86885', 'LETICIA': '91001', 'EL ENCANTO': '91263', 'LA CHORRERA': '91405', 'LA PEDRERA': '91407', 'MIRITI - PARANA': '91460', 'PUERTO ALEGRIA': '91530', 'PUERTO ARICA': '91536', 'PUERTO NARINO': '91540', 'TARAPACA': '91798', 'INIRIDA': '94001', 'BARRANCO MINAS': '94343', 'MAPIRIPANA': '94663', 'SAN FELIPE': '94883', 'LA GUADALUPE': '94885', 'CACAHUAL': '94886', 'PANA PANA': '94887', 'MORICHAL': '94888', 'SAN JOSE DEL GUAVIARE': '95001', 'EL RETORNO': '95025', 'MITU': '97001', 'CARURU': '97161', 'PACOA': '97511', 'TARAIRA': '97666', 'PAPUNAUA': '97777', 'YAVARATE': '97889', 'PUERTO CARRENO': '99001', 'LA PRIMAVERA': '99524', 'SANTA ROSALIA': '99624', 'CUMARIBO': '99773'}

''' def load_image(image_path):
    image = Image.open(image_path)
    return image '''

def extract_info_image(image_path):
    args = {
    "image": image_path,  # Establecer URL de la imagen
    #"preprocess": "gamma_correction"  # Seleccionar preprocesamiento
    }

    # load the example image and convert it to grayscale
    image = cv2.imread(args["image"])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, gray)

    text = pytesseract.image_to_string(Image.open(filename))

    patrones = {
        "pin": r"Pin\s*No:\s*(\d+)",
        "matricula": r"Nro\s*Matricula:\s*([\d-]+)",
        "municipio": r"MUNICIPIO:\s*([A-Z\s]+)(?:\s+VEREDA:|\s+[A-Z]+:)",
        "codigo_catastral": r"CODIGO\s*CATASTRAL:\s*(\d+)"
    }

    # Buscar los patrones en el texto
    image_info = {}
    for clave, patron in patrones.items():
        coincidencias = re.search(patron, text)
        if coincidencias:
            image_info[clave] = coincidencias.group(1)

    # Mostrar la información obtenida
    print("Información extraída:")
    print(image_info)
    os.remove(filename)
    return image_info

def query_requests(image_info):
    base_url = 'https://serviciosgeovisor.igac.gov.co:8080/Geovisor/catastral'

    consulta = f"{data[image_info['municipio']]}0000{image_info['codigo_catastral']}000000"

    params = {
        'cmd': 'query_codigo',
        't': '1718137312033'
    }

    payload = {
        'query': consulta # Código del predio
    }

    try:
        response = requests.post(base_url, params=params, data=payload)

        if response.status_code == 200:
            json_data = response.json()

    except requests.exceptions.HTTPError as http_err:
        print(f'Error HTTP: {http_err}')
    except requests.exceptions.RequestException as req_err:
        print(f'Error en la solicitud: {req_err}')
    except ValueError as json_err:
        print(f'Error al parsear JSON: {json_err}')



    query_folder = f"{payload['query']}"
    if not os.path.exists(query_folder):
        os.makedirs(query_folder)

    # Obtener los anillos de la geometría dentro de la clave "geometry"
    rings = json_data['response']['terreno']['features'][0]['geometry']['rings']

    # Convertir los anillos en polígonos
    polygons = [Polygon(ring) for ring in rings]

    # Crear un GeoDataFrame con los polígonos
    gdf = gpd(geometry=polygons)

    bbox = gdf.total_bounds.tolist() # Calcular el bbox en base al GeoDataFrame

    # Guardar el GeoDataFrame como Shapefile
    shapefiles_basename = f"{query_folder}/{payload['query']}" # Este nos permite establecer el nombre de los 4 archivos

    shapefile_filename = f"{shapefiles_basename}.shp" # Este nos permite suministrarlo al gdf para generar los archivos sin crear una nueva carpeta
    gdf.to_file(shapefile_filename)


    geojson_filename = f"{payload['query']}.geojson"
    gdf.to_file(geojson_filename, driver='GeoJSON')

    with open(geojson_filename, 'r+') as f:
        geojson_data = json.load(f)
        geojson_data['bbox'] = bbox # Añadir bbox al archivo GeoJSON
        f.seek(0)  # Moverse al inicio del archivo
        json.dump(geojson_data, f, indent=4)
        f.truncate()  # Truncar el archivo si es necesario


    # Añadir los rings y bbox al diccionario image_info
    image_info['rings'] = rings
    image_info['bbox'] = bbox

    # Guardar la image_info en un nuevo archivo JSON
    json_filename = f"{payload['query']}.json"
    with open(json_filename, 'w') as outfile:
        json.dump(image_info, outfile, indent=4)


    # Crear un archivo ZIP y añadir los archivos generados
    zip_filename = f"{payload['query']}.zip"
    with ZipFile(zip_filename, 'w') as zipf:

    # Añadir los archivos del shapefile
        for ext in ['shp', 'shx', 'dbf', 'cpg']:
            zipf.write(f"{shapefiles_basename}.{ext}")
    # Eliminar la carpeta después de comprimir los archivos
    rmtree(query_folder)

    print(f"Todos los archivos han sido comprimidos en '{zip_filename}'.")
    # return zip_filename, json_filename, geojson_filename


path_image = 'certificado.jpg'
query_requests(extract_info_image(path_image))

Información extraída:
{'pin': '220524435259585518', 'matricula': '300-80832', 'municipio': 'LEBRIJA', 'codigo_catastral': '000000110099000'}
Todos los archivos han sido comprimidos en '684060000000000110099000000000.zip'.
